# ComfyUI for Paperspace - 完全修正版

**🎨 変数展開エラーを完全修正した安定版**

## ✨ 修正内容
- **変数展開エラーの完全解決**: `{COMFY_DIR}` → 正しいパス展開
- **xformers自動整合**: PyTorchバージョンに最適なxformersを自動選択
- **誤ディレクトリ自動削除**: 過去の失敗で作成された不正ディレクトリを自動クリーンアップ
- **HuggingFaceトークン対応**: 認証が必要なモデルの安全なダウンロード
- **Google Drive対応**: `gdown --fuzzy`で柔軟な共有リンク処理
- **Paperspace最適化**: 永続ストレージとポート設定の最適化

## 🚀 使用方法
1. PaperspaceでPyTorchテンプレートを選択
2. このノートブックを実行
3. 「Open Port 8123」でComfyUIにアクセス

## 📁 保存先
- ComfyUI本体: `/notebooks/ComfyUI`
- 生成画像: `/notebooks/ComfyUI/output`（永続保存）
- モデル: `/notebooks/ComfyUI/models/checkpoints`

In [ ]:
# ================================================================
#  ComfyUI for Paperspace - Complete Fixed Edition
# ================================================================
import os
import sys
import subprocess
from pathlib import Path
import time

print("🔧 ComfyUI Paperspace 完全修正版セットアップ開始")
print("=" * 60)

# -------------------------
# 基本パスの定義（pathlibで堅牢に）
# -------------------------
COMFY_DIR = Path('/notebooks/ComfyUI')
MGR_DIR = COMFY_DIR / 'custom_nodes' / 'ComfyUI-Manager'
CHECKPOINT_DIR = COMFY_DIR / 'models' / 'checkpoints'
OUTPUT_DIR = COMFY_DIR / 'output'
INPUT_DIR = COMFY_DIR / 'input'
PORT = 8123

# 必要ディレクトリの事前作成
for directory in [CHECKPOINT_DIR, OUTPUT_DIR, INPUT_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# -------------------------
# 誤ディレクトリの完全クリーンアップ
# -------------------------
print("🧹 過去の変数展開エラーで作成された誤ディレクトリをクリーンアップ中...")
wrong_dirs = [
    Path('{COMFY_DIR}'), Path('{MGR_DIR}'), Path('{CHECKPOINT_DIR}'),
    Path('/notebooks/{COMFY_DIR}'), Path('/notebooks/{MGR_DIR}'),
    Path('ComfyUI'),  # カレントディレクトリの誤クローン
    Path('/content/ComfyUI'),  # Colab環境の残骸
]

cleaned_count = 0
for wrong in wrong_dirs:
    if wrong.exists():
        print(f"  🗑️ 削除中: {wrong}")
        subprocess.run(['rm', '-rf', str(wrong)], check=False, capture_output=True)
        cleaned_count += 1

if cleaned_count > 0:
    print(f"✅ {cleaned_count}個の誤ディレクトリを削除しました")
else:
    print("✅ クリーンアップ不要（誤ディレクトリなし）")

# Step-1: 環境の準備
print("\n■ Step-1: 環境の準備をしています...")
subprocess.run(['apt-get', '-yqq', 'update'], check=False, capture_output=True)
subprocess.run([
    'apt-get', '-yqq', 'install', 
    'git', 'wget', 'build-essential', 'cmake', 
    'libgl1', 'libglib2.0-0', 'ffmpeg', 'git-lfs'
], check=False, capture_output=True)
subprocess.run(['git', 'lfs', 'install'], check=False, capture_output=True)

# Step-2: ComfyUIとManagerのセットアップ
print("\n■ Step-2: ComfyUIとManagerをセットアップしています...")

# ComfyUI本体のセットアップ
if not COMFY_DIR.exists():
    print(f"📥 ComfyUIを新規インストール中: {COMFY_DIR}")
    subprocess.run([
        'git', 'clone', '--depth=1', 
        'https://github.com/comfyanonymous/ComfyUI.git', 
        str(COMFY_DIR)
    ], check=True)
else:
    print(f"🔄 既存ComfyUIを更新中: {COMFY_DIR}")
    subprocess.run(['git', 'pull', '--rebase', '--autostash'], 
                   cwd=str(COMFY_DIR), check=False)

# ComfyUI-Managerのセットアップ
if not MGR_DIR.exists():
    print(f"📥 ComfyUI-Managerを新規インストール中: {MGR_DIR}")
    subprocess.run([
        'git', 'clone', '--depth=1',
        'https://github.com/ltdrdata/ComfyUI-Manager.git',
        str(MGR_DIR)
    ], check=True)
else:
    print(f"🔄 既存ComfyUI-Managerを更新中: {MGR_DIR}")
    subprocess.run(['git', 'pull', '--rebase', '--autostash'], 
                   cwd=str(MGR_DIR), check=False)

# Step-3: 依存関係のインストール
print("\n■ Step-3: 依存関係をインストールしています...")

# 基本パッケージの更新
subprocess.run([
    sys.executable, '-m', 'pip', 'install', '-q', '--upgrade',
    'pip', 'wheel', 'setuptools', 'gdown', 'psutil'
], check=False)

# 基本科学計算ライブラリ
subprocess.run([
    sys.executable, '-m', 'pip', 'install', '-q', '--upgrade',
    'numpy', 'scipy', 'Pillow', 'toml'
], check=False)

# ComfyUI要件のインストール
comfy_req = COMFY_DIR / 'requirements.txt'
if comfy_req.exists():
    print(f"🔧 ComfyUI依存関係をインストール中: {comfy_req}")
    subprocess.run([
        sys.executable, '-m', 'pip', 'install', '-q',
        '-r', str(comfy_req), '--prefer-binary'
    ], check=False)

# Manager要件のインストール（エラー許容）
mgr_req = MGR_DIR / 'requirements.txt'
if mgr_req.exists():
    print(f"🔧 ComfyUI-Manager依存関係をインストール中: {mgr_req}")
    try:
        subprocess.run([
            sys.executable, '-m', 'pip', 'install', '-q',
            '-r', str(mgr_req), '--prefer-binary'
        ], check=True, timeout=300)
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired):
        print("⚠️ Manager要件の一部でエラーまたはタイムアウトが発生しましたが続行します")

# xformersのPyTorchバージョン自動整合
print("\n■ Step-3b: xformersをPyTorchバージョンに自動整合中...")
try:
    import torch
    torch_ver = torch.__version__.split('+')[0]
    major_minor = '.'.join(torch_ver.split('.')[:2])
    
    # PyTorchバージョン別のxformers推奨マッピング
    xformers_map = {
        '2.0': '0.0.20',
        '2.1': '0.0.22.post7',
        '2.2': '0.0.25',
        '2.3': '0.0.27',
        '2.4': '0.0.28.post1',
        '2.5': '0.0.28.post1',
    }
    
    xformers_ver = xformers_map.get(major_minor, '0.0.28.post1')
    print(f"🔧 PyTorch {torch_ver} → xformers {xformers_ver} をインストール中...")
    
    subprocess.run([
        sys.executable, '-m', 'pip', 'install', '-q',
        f'xformers=={xformers_ver}', '--prefer-binary'
    ], check=False, timeout=300)
    
    print("✅ xformers自動整合完了")
    
except Exception as e:
    print(f"⚠️ xformers自動整合に失敗: {e}")
    print("🔄 フォールバック: 最新版xformersをインストール中...")
    subprocess.run([
        sys.executable, '-m', 'pip', 'install', '-q',
        'xformers', '--prefer-binary'
    ], check=False)

# Step-4: 基本モデルのダウンロード
print("\n■ Step-4: 基本モデルのダウンロード...")
model_file = CHECKPOINT_DIR / 'v1-5-pruned-emaonly.safetensors'
HF_TOKEN = os.environ.get('HUGGINGFACE_TOKEN', '').strip()
HF_URL = 'https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors'

if not model_file.exists():
    print("📥 Stable Diffusion 1.5をダウンロード中...")
    
    download_success = False
    
    if HF_TOKEN:
        print("🔐 HuggingFaceトークンを使用して認証ダウンロード")
        try:
            result = subprocess.run([
                'wget', '-q', '-O', str(model_file),
                '--header', f'Authorization: Bearer {HF_TOKEN}',
                HF_URL
            ], check=True, timeout=600)
            download_success = True
            print("✅ 認証付きダウンロード完了")
        except (subprocess.CalledProcessError, subprocess.TimeoutExpired):
            print("❌ 認証付きダウンロード失敗")
    
    if not download_success:
        print("🔄 トークンなしでダウンロードを試行中...")
        try:
            result = subprocess.run([
                'wget', '-c', HF_URL, '-O', str(model_file)
            ], check=True, timeout=600)
            download_success = True
            print("✅ モデルダウンロード完了")
        except (subprocess.CalledProcessError, subprocess.TimeoutExpired):
            print("❌ モデルダウンロード失敗（403エラーまたはタイムアウト）")
            print("💡 手動でモデルを配置するか、HuggingFaceトークンを設定してください")
            
    # ダウンロードファイルの検証
    if model_file.exists() and model_file.stat().st_size > 1000000:  # 1MB以上
        size_gb = model_file.stat().st_size / (1024**3)
        print(f"✅ モデルファイル確認完了 ({size_gb:.2f}GB)")
    else:
        print("⚠️ モデルファイルが不完全または存在しません")
else:
    print("✅ 基本モデルは既に存在します")

# Step-5: Google Driveからの追加モデル（オプション）
print("\n■ Step-5: Google Driveからの追加モデル（オプション）")
gdrive_url = "ここにGoogleドライブの共有リンクを貼り付けてください"
output_filename = "custom_model.safetensors"

if "drive.google.com" in gdrive_url and "ここに" not in gdrive_url:
    print(f"📥 Google Driveから {output_filename} をダウンロード中...")
    try:
        subprocess.run([
            sys.executable, '-m', 'gdown', '--fuzzy', gdrive_url,
            '-O', str(CHECKPOINT_DIR / output_filename)
        ], check=True, timeout=600)
        print("✅ Google Driveダウンロード完了")
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired):
        print("❌ Google Driveダウンロード失敗")
        print("💡 共有リンクの権限設定を確認してください")
else:
    print("💡 Google Driveダウンロードをする場合は、上記URLを実際のリンクに変更してください")

# Step-6: ComfyUIの起動
print("\n" + "=" * 60)
print("✅ セットアップ完了！ComfyUIを起動します")
print(f"🌐 アクセス方法: Paperspaceの 'Open Port {PORT}' ボタンをクリック")
print(f"📁 出力先: {OUTPUT_DIR}")
print(f"🛑 停止方法: Ctrl+C または ランタイム切断")
print("=" * 60)

# 環境変数でメモリ使用量を最適化
env = os.environ.copy()
env['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# ComfyUI起動コマンド（Paperspace最適化）
startup_cmd = [
    sys.executable, 'main.py',
    '--listen=0.0.0.0',  # Paperspaceからのアクセスを許可
    f'--port={PORT}',
    f'--output-directory={OUTPUT_DIR}',  # 永続ストレージに保存
    '--temp-directory=/tmp',  # 一時ファイルはtmpに
    '--disable-cuda-malloc',  # メモリ最適化
    '--normalvram'  # VRAM使用量抑制
]

print(f"🚀 ComfyUI起動中...")
print(f"💻 実行コマンド: {' '.join(startup_cmd)}")

try:
    subprocess.run(startup_cmd, cwd=str(COMFY_DIR), env=env, check=False)
except KeyboardInterrupt:
    print("\n🛑 ComfyUIを停止しました")
except Exception as e:
    print(f"❌ ComfyUI起動エラー: {e}")
    print("💡 Paperspaceランタイムを再起動してください")

## 🔧 トラブルシューティング

### **HuggingFaceトークンの設定**
```python
import os
# あなたのトークンに変更してください
os.environ['HUGGINGFACE_TOKEN'] = 'hf_xxxxxxxxxxxxxxxxx'
# その後、上記メインセルを再実行
```

### **手動モデル配置**
```python
# モデルディレクトリの確認
!ls -la /notebooks/ComfyUI/models/checkpoints/
# ここに .safetensors ファイルを手動アップロード
```

### **プロセス確認・強制終了**
```python
# ComfyUIプロセスの確認
!ps aux | grep main.py | grep -v grep
# ポートの使用状況確認
!netstat -tlnp | grep 8123
# プロセス強制終了
!pkill -f "python main.py"
!fuser -k 8123/tcp
```

### **xformers問題の手動解決**
```python
# PyTorchバージョン確認
import torch
print(f"PyTorch: {torch.__version__}")

# xformers手動再インストール
!pip uninstall -y xformers
!pip install xformers==0.0.27  # PyTorch 2.3系の場合
!pip install xformers==0.0.28.post1  # PyTorch 2.4系の場合
```

### **完全リセット**
```python
# ComfyUIを完全削除して再インストール
!rm -rf /notebooks/ComfyUI
# その後、メインセルを再実行
```

### **メモリ不足対策**
```python
# より軽量な起動オプション
# メインセルの起動コマンドに以下を追加：
# '--lowvram'  # VRAM使用量をさらに削減
# '--cpu'      # CPU推論モード（非常に遅い）
```